Task 2

In [ ]:
# downloading libraries
!pip install mysql-connector-python
!pip install pymysql sqlalchemy retrying
# import libraries
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import mysql.connector
from numpy import short
import pymysql
from sqlalchemy import create_engine
from retrying import retry


1.	Read the csv or excel file.

In [ ]:
data = pd.read_csv("/content/used_bikes.csv")

In [ ]:
# memory info
data.memory_usage(deep=True)

Index           128
bike_name     11954
price          1192
city           9546
kms_driven     1192
owner         10144
age            1192
power          1192
brand          9517
dtype: int64

In [ ]:
memory_before = data.memory_usage(deep=True).sum()

In [ ]:
# lets have a look at the data ...
data.sample(7)

,bike_name,price,city,kms_driven,owner,age,power,brand
49,Suzuki Gixxer SF Fi 150cc SP ABS,97000.0,Delhi,4400.0,First Owner,3.0,150.0,Suzuki
88,Yamaha FZ16 150cc,50000.0,Bangalore,19000.0,First Owner,6.0,150.0,Yamaha
73,Royal Enfield Thunderbird X 350cc,150000.0,Bangalore,15000.0,First Owner,3.0,350.0,Royal Enfield
5,Yamaha FZs 150cc,53499.0,Delhi,25000.0,First Owner,6.0,150.0,Yamaha
77,Royal Enfield Thunderbird 350cc,115000.0,Bangalore,23700.0,First Owner,4.0,350.0,Royal Enfield
74,Hero Karizma ZMR 223cc,55786.0,Surat,68000.0,Second Owner,7.0,223.0,Hero
148,Yamaha YZF-R15 V3 150cc,140000.0,Delhi,2473.0,First Owner,2.0,150.0,Yamaha


2.	Change the datatypes of the column and check the memory usage before and after the change in the data types.

In [ ]:
# lets see maximum values so we can reduced data types
print("age:",data["age"].max(),"power :",data.power.max(),"price: ",data.price.max(),"kms_driven:",data.kms_driven.max(),sep="\n")

age:
16.0
power :
1299.0
price: 
1350000.0
kms_driven:
357000.0


In [ ]:
# lets see there current data types...
data.dtypes

bike_name      object
price         float64
city           object
kms_driven    float64
owner          object
age           float64
power         float64
brand          object
dtype: object

In [ ]:
# convert age to byte and power to short
convert = {'age':bytes,'power':short}
data = data.astype(convert)
data.dtypes

bike_name      object
price         float64
city           object
kms_driven    float64
owner          object
age              |S32
power           int16
brand          object
dtype: object

In [ ]:
# check memory usage
print("Memory reduced = ",abs(memory_before-data.memory_usage(deep=True).sum()))

Memory reduced =  2682


In [ ]:
# we can reduce this further by several methods like ---

# 1. by removing brand name from bike name

# 2. by using abbreviations for cities like del for delhi

# 3. encoding owners

In [ ]:
data.to_csv("now.csv",index=False)

3.	Dump the data into the mysql database.


4.	index the column after the data is inserted.


In [ ]:
# sql connect...
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Emmabhi@15",
    database="neenopal"
)

cursor = mydb.cursor()

# Create Database neenopal...
cursor.execute("CREATE DATABASE neenopal")

# Create Table used_bikes...
cursor.execute("create table used_bikes(bike_name varchar(50),"
               "price float8,"
               "city varchar(20),"
               "kms_driven float8,"
               "owner varchar(20),"
               "age int,power int,"
               "brand varchar(20))")


table_name = 'used_bikes'
column_name = 'bike_name'


cursor.execute(f"SHOW INDEX FROM {table_name} WHERE Key_name='idx_{column_name}'")
index_exists = cursor.fetchall()

# drop index
if index_exists:
    cursor.execute(f"DROP INDEX idx_{column_name} ON {table_name}")

# inserting data
data_tuples = [tuple(row) for row in data.values]


sql_insert = f"INSERT INTO {table_name} (bike_name, price, city, kms_driven, owner, age, power, brand) VALUES (%s, " \
             f"%s, %s, %s, %s, %s, %s, %s) "


cursor.executemany(sql_insert, data_tuples)

# inserting index
cursor.execute(f"CREATE INDEX idx_{column_name} ON {table_name}({column_name})")
cursor.close()
mydb.close()